In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.draw import draw_squares
from utils.square import SquareDataset
from torch.utils.data import DataLoader

In [ ]:
class LinearModel(nn.Module):
    def __init__(self, x, y):
        super(LinearModel, self).__init__()
        self.layer1 = nn.Linear(x, y)

    def forward(self, x):
        x = self.layer1(x)
        return F.softmax(x, dim=1)

In [ ]:
squares = SquareDataset(3000)
for i in range(5):
    print(squares[i])

# Setting things up!

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Use the nn package to define our model and loss function.
model = LinearModel(9, 3)
model = model.to(device)

cost = torch.nn.BCELoss()

# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# dataset!
dataloader = DataLoader(squares, batch_size=128)

epochs = 350

# The Optimization Loop

In [ ]:
for t in range(epochs):
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device) / 255, Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

    if t % 50 == 0:
        print('l: {:>8f}, (e {:>3})'.format(loss.item(), t))
        
print('l: {:>8f}, (e {:>3})\nDone!'.format(loss.item(), t))

In [ ]:
print("W's and b's:")
for p in model.parameters():
    print(p)

# Trying it out (inference)

In [ ]:
img = [255, 255, 145, 255, 255, 255, 232, 32, 255]
with torch.no_grad():
    o = model(torch.tensor(img, dtype=torch.float).view(1, 9).to(device) / 255)
    
print(o)
print('\ntop:    {:>8f}\nmiddle: {:>8f}\nbottom: {:>8f}'.format(*o[0]))